<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# EPAT Session 2

**Executive Program in Algorithmic Trading**

**_Event-Based Backtesting_**

Prof. Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

<a href="https://home.tpq.io/certificates/pyalgo" target="_blank"><img src="https://hilpisch.com/pyalgo_cover_color.png" width="300px" align="left" border="1px"></a>

## Basic Imports

In [1]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn')
pd.set_option('mode.chained_assignment', None)

## Financial Data Class

In [2]:
url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'  # EOD data
# url = 'http://hilpisch.com/aiif_eikon_id_data.csv'  # intraday data

In [3]:
raw = pd.read_csv(url, index_col=0, parse_dates=True).dropna()

In [4]:
raw.columns

Index(['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N', 'SPY', '.SPX', '.VIX',
       'EUR=', 'XAU=', 'GDX', 'GLD'],
      dtype='object')

In [5]:
class FinancialData:
    url = 'http://hilpisch.com/pyalgo_eikon_eod_data.csv'
    def __init__(self, symbol):
        self.symbol = symbol
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0,
                               parse_dates=True).dropna()
    def prepare_data(self):
        self.data = pd.DataFrame(self.raw[self.symbol])
        self.data['r'] = np.log(self.data[self.symbol] /
                                self.data[self.symbol].shift(1))
        # self.data.dropna(inplace=True)

In [6]:
fd = FinancialData('EUR=')

## Event-Based Approach

In [7]:
import time

In [8]:
for bar in range(10):
    print(bar)
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9


In [9]:
for bar in range(10):
    print(bar, fd.data.index[bar], fd.data[fd.symbol].iloc[bar])
    time.sleep(1)

0 2010-01-04 00:00:00 1.4411
1 2010-01-05 00:00:00 1.4368
2 2010-01-06 00:00:00 1.4412
3 2010-01-07 00:00:00 1.4318
4 2010-01-08 00:00:00 1.4412
5 2010-01-11 00:00:00 1.4513
6 2010-01-12 00:00:00 1.4494
7 2010-01-13 00:00:00 1.451
8 2010-01-14 00:00:00 1.4502
9 2010-01-15 00:00:00 1.4382


## Event-Based Backtesting (Base Class)

The following methods are getting implemented:

    .get_date_price()
    .print_current_balance()
    .print_net_wealth()
    .place_buy_order()
    .place_sell_order()
    .close_out()

In [10]:
class BacktestBase(FinancialData):
    def __init__(self, symbol, amount):
        super().__init__(symbol)
        self.initial_amount = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
    def get_date_price(self, bar):
        date = str(self.data.index[bar])[:10]
        price = self.data[self.symbol].iloc[bar]
        return date, price
    def print_current_balance(self, bar):
        date, price = self.get_date_price(bar)
        print(f'{date} | current balance = {self.current_balance:.2f}')
    def print_net_wealth(self, bar):
        date, price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units * price
        print(f'{date} | net wealth = {net_wealth:.2f}')
    def place_buy_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance -= units * price
        self.units += units
        self.trades += 1
        print(f'{date} | bought {units} at price {price}')
    def place_sell_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / price)
        self.current_balance += units * price
        self.units -= units
        self.trades += 1
        print(f'{date} | sold {units} at price {price}')
    def close_out(self, bar):
        date, price = self.get_date_price(bar)
        print(55 * '=')
        print(f'{date} | CLOSING OUT POSITION')
        print(55 * '=')
        self.current_balance += self.units * price
        perf = (self.current_balance - self.initial_amount) / self.initial_amount
        print(f'{date} | closing {self.units} at price {price}')
        self.units = 0
        self.print_current_balance(bar)
        self.print_net_wealth(bar)
        print(f'{date} | net performance [%] = {perf:.3f}')
        print(f'{date} | trades [#] = {self.trades}')

In [11]:
bb = BacktestBase('EUR=', 10000)

In [12]:
bb.get_date_price(100)

('2010-05-27', 1.2368)

In [13]:
bb.print_current_balance(150)

2010-08-09 | current balance = 10000.00


In [14]:
bb.print_net_wealth(150)

2010-08-09 | net wealth = 10000.00


In [15]:
bb.place_buy_order(200, amount=2500)

2010-10-19 | bought 1820 at price 1.3732


In [16]:
bb.print_current_balance(200)

2010-10-19 | current balance = 7500.78


In [17]:
bb.print_net_wealth(200)

2010-10-19 | net wealth = 10000.00


In [18]:
bb.place_sell_order(500, units=820)

2011-12-27 | sold 820 at price 1.3068


In [19]:
bb.print_current_balance(500)

2011-12-27 | current balance = 8572.35


In [20]:
bb.print_net_wealth(500)

2011-12-27 | net wealth = 9879.15


In [21]:
bb.close_out(bar=1200)

2014-10-09 | CLOSING OUT POSITION
2014-10-09 | closing 1000 at price 1.269
2014-10-09 | current balance = 9841.35
2014-10-09 | net wealth = 9841.35
2014-10-09 | net performance [%] = -0.016
2014-10-09 | trades [#] = 2


## Event-Based Backtesting (SMA Backtester)

In [22]:
class SMABacktester(BacktestBase):
    def prepare_statistics(self, SMA1, SMA2):
        self.data['SMA1'] = self.data[self.symbol].rolling(SMA1).mean()
        self.data['SMA2'] = self.data[self.symbol].rolling(SMA2).mean()
    def backtest_strategy(self, SMA1, SMA2):
        print(f'BACKTESTING SMA1={SMA1} | SMA2={SMA2}')
        print(55 * '=')
        self.prepare_statistics(SMA1, SMA2)
        self.units = 0
        self.trades = 0
        self.position = 0
        self.current_balance = self.initial_amount
        self.data['signal'] = np.where(self.data['SMA1'] > self.data['SMA2'], 1, -1)      
        for bar in range(SMA2, len(self.data)):
            signal = self.data['signal'].iloc[bar]
            # add more logic (checking for SL, TP, TSL)
            if self.position in [0, -1] and signal == 1:
                # add more logic
                self.place_buy_order(bar, units=(1 - self.position) * 1000)
                self.print_current_balance(bar)
                self.print_net_wealth(bar)
                self.position = 1
                print(55 * '=')
            elif self.position in [0, 1] and signal == -1:
                # add more logic
                self.place_sell_order(bar, units=(1 + self.position) * 1000)
                self.print_current_balance(bar)
                self.print_net_wealth(bar)
                self.position = -1
                print(55 * '=')
        self.close_out(bar)

In [23]:
sma = SMABacktester('EUR=', 10000)

In [24]:
sma.backtest_strategy(42, 252)

BACKTESTING SMA1=42 | SMA2=252
2011-01-03 | bought 1000 at price 1.3351
2011-01-03 | current balance = 8664.90
2011-01-03 | net wealth = 10000.00
2011-10-07 | sold 2000 at price 1.3375
2011-10-07 | current balance = 11339.90
2011-10-07 | net wealth = 10002.40
2012-11-01 | bought 2000 at price 1.2942
2012-11-01 | current balance = 8751.50
2012-11-01 | net wealth = 10045.70
2014-07-22 | sold 2000 at price 1.3464
2014-07-22 | current balance = 11444.30
2014-07-22 | net wealth = 10097.90
2016-03-17 | bought 2000 at price 1.1317
2016-03-17 | current balance = 9180.90
2016-03-17 | net wealth = 10312.60
2016-11-08 | sold 2000 at price 1.1023
2016-11-08 | current balance = 11385.50
2016-11-08 | net wealth = 10283.20
2017-06-02 | bought 2000 at price 1.128
2017-06-02 | current balance = 9129.50
2017-06-02 | net wealth = 10257.50
2018-06-14 | sold 2000 at price 1.1567
2018-06-14 | current balance = 11442.90
2018-06-14 | net wealth = 10286.20
2019-12-31 | CLOSING OUT POSITION
2019-12-31 | closing

<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>